In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as tf
import cv2 as cv
from google.colab.patches import cv2_imshow
from collections import OrderedDict
import cv2 as cv
from PIL import Image

In [218]:
class Net(nn.Module):
   def __init__(self):
    super().__init__()
    #PRETRAINED MODEL
    self.pretrained = models.vgg19(weights="DEFAULT")
    for param in self.pretrained.parameters():
      param.requires_grad = False
    #all available hooks
    self.fhooks = []
    # initialising hooks
    self.fhooks.append(self.pretrained._modules['classifier'][1].register_forward_hook(self.hookfn))
    ##self.fhooks.append(self.pretrained._modules['classifier'][1].register_forward_hook(self.hookfn))
    #store hooks output
    self.selected_out = OrderedDict()
    self.count=0
    self.limit=1 # layers limit ,i.e for how many layer i want the output

   def forward(self,x):
    y=self.pretrained(x)

    return y,self.selected_out

   def hookfn(self,layername,inp,out):
    self.selected_out[self.count]=out
    #print(self.count,str(layername)) ,one can try to see in which order these are getting printed
    self.count+=1
    if self.limit==self.count:
      self.count=0



In [198]:
def l2_loss(current,actual):
  return torch.norm(current-actual)/(torch.norm(actual)); #by default forbenius norm or l2-norm

def variation_loss(current):
  #variation within assumed image.
  dx = torch.sum(torch.pow((current[:,:,:,1:] - current[:,:,:,:-1]), 2))
  dy = torch.sum(torch.pow((current[:,:,1:,:] - current[:,:,:-1,:]), 2))
  #print(dx.shape)
  #print(dy.shape)
  loss = dx+dy
  return loss


In [199]:
transform = tf.Compose(
    [tf.ToTensor(),
    tf.Normalize(mean=[0.485, 0.456, 0.406],  #normalize image with the mean and standard deviation used for training vgg19,bgr mode
                  std=[0.229, 0.224, 0.225]),
    tf.CenterCrop(size=(256,256))]
)

In [200]:
import copy

In [201]:

def recreate_image(img):
    '''
    normalise the image again to be able to display
    '''
    reverse_mean = [-0.485, -0.456, -0.406] #rgb
    reverse_std = [1/0.229, 1/0.224, 1/0.225] #rgb
    img=img.cpu()
    recreated_im = copy.copy(img.data.numpy()[0])
    for c in range(3):
        recreated_im[c] /= reverse_std[c]
        recreated_im[c] -= reverse_mean[c]
    recreated_im[recreated_im > 1] = 1 # clipping ...
    recreated_im[recreated_im < 0] = 0 #clipping
    recreated_im = np.round(recreated_im * 255)

    recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)

    #con_image=np.uint8(image*255.0)
    con_image=cv.cvtColor(recreated_im, cv.COLOR_RGB2BGR) # rgb->bgr , since cv2_imshow assumes image is in bgr format
    cv2_imshow(con_image)
    return

In [231]:
#not creating a function for the current instance .
 # inference mode , to avoid gradient calculation for the model variables
def optimize(model,device,lr=1e3,momentum=0.9,iterations=300,step_size=40,gamma=0.1):

    x=torch.randn((1,3,256,256))*0.1 # since we want to optimize input not model
    print(x.is_leaf)
    img=cv.imread('/content/pic10.jpg') #read as BGR
    img=np.float32(cv.cvtColor(img, cv.COLOR_BGR2RGB)) #my output is in rgb mode
    img/=255
    torch_img=transform(img)
    img=torch_img.unsqueeze(0) # adding an extra dimension as axis=0.

    model=model.to(device)
    img=img.to(device)
    x=x.to(device)
    x.requires_grad=True
    print(x.is_leaf)
    lr=lr
    momentum=momentum

    model.eval()
    #model.requires_grad(False)

    _,embed_out=model(img)

    embed=embed_out[0]

    total_loss=0.0
    iterations=iterations

    optimizer=torch.optim.SGD([x],lr = lr, momentum=momentum)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)

    for i in range(iterations):

      optimizer.zero_grad()

      _,hookout=model(x)
      out=hookout[0]

      l2norm=l2_loss(out,embed)*1e-1
      tvbregularise=variation_loss(x)*1e-5
      #print(l2norm,tvbregularise)
      total_loss=l2norm+tvbregularise

      total_loss.backward(retain_graph=True)

      optimizer.step()
      #scheduler.step()

      #total_loss.backward(retain_graph = True)

      if i%20==19:
        print(i,total_loss)
        recreate_image(x)

    return x#.cpu().detach().numpy()

In [232]:
model=Net()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")



In [ ]:
t=optimize(model,device)